### 내가 구사하려는 문장과 비슷한 것들을 많이 크롤링해야 조타

### 마르코프 체인으로 문장 생성하기
- 마르코프 체인은 확률을 기반으로 문장을 이어 붙여 나가는 방법
- 워드 샐러드라고도 불림

#### 문장 제작 과정
1) 문장을 단어로 분할(형태소 분석)한다 - 영어는 nlpy, 한글은 konlpy
2) 단어의 전후 연결을 딕셔너리에 등록한다
3) 사전을 사용해 임의의 문장을 생성한다 - 딕셔너리에 등록된 단어들을 이어 붙여 나가는 문장을 생성한다

#### 문장제작 예시
- "그는 고양이를 좋아합니다."
- 위 문장을 형태소로 문석하면 ["그","는", "고양이","를", "좋아", "합니다"]
- 각 단어의 전후 관계를 알 수 있게 3개의 요소씩 묶어 사전으로 등록한다.

- 그+는+고양이
- 는+고양이+를
- 고양이+를+좋아
- 를+좋아+합니다

- 단어와 단어를 처리하고 이를 기반으로 단어들을 연결하여 문장을 생성

In [3]:
import markov
from konlpy.tag import Okt
import json

In [4]:
path = 'data/홍길동전.txt'
with open(path, 'r', encoding='utf-8') as f:
  text = str(f.readlines())

In [7]:
twitter = Okt()
lines = text.split('\n')
words = []

for line in lines:
  malist = twitter.pos(line, norm=True, stem=True)
  for word in malist:
    if word[1] not in ['Punctuation']:
      words.append(word[0])
    if word[0] == '.':
      words.append(word[0]) # Punctuation 안에 .이 있으면 추가해줘야함(물음표 쌍따옴표 뺌)

In [8]:
# 딕셔너리 생성
dic = markov.make_dic(words)
json.dump(dic, open('data/hkt.json', 'w', encoding='utf-8'))

---
## 문장 생성하기

In [10]:
# 딕셔너리 불러오기
dic = json.load(open('data/hkt.json', 'r', encoding='utf-8'))

In [18]:
# 문장 만들기
for i in range(3):
  print(markov.make_sentence(dic))
  print('s')
  print('-----')

심다 의혹 하다 괴이 한 말 을 다시 볼 날 이 밝다 소식 없다 음 을 어찌 하리 요 네 일찍 돌아오다 바라다 n 하다 계 하 에 정조 삼천 석 을 무릅쓰 고 죽다 되다 그 부형 되다 자가 그 같다 고명 한 자제 를 두다 하다 이제 여덟 이 되어다 거리 로 다니다 방 을 붙이다 이르다 n 즉금 우리 의논 이 분운 하다 수문 장이 급 고 왈 n 그 아이 거동 이 비범하다 뿐 아니다 .
s
-----
대성통곡 후 에 그날 부터 중문 밖에 나다 아니다 행실 을 닦다 그 달 부터 태기 있다 십 삭 이 당하다 거처 하다 방 에 들어오다 재물 을 노략 하다 혹 불쌍하다 백성 을 구제 하다 일후 총 을 놓다 잡다 n 하다 즉시 관상 녀 의 말씀 같이 왕기 있다 만일 범람 한 마음 도리어 황공 하다 팔십 노친 이 백 수 모년 에 너 로 인하다 세상 을 속 이고 그 재물 을 욕심 하다 사지 에 빠지다 줄 알 지 말고 보이다 대로 기이 말르다 n 하다 너 를 잡다 하옵다 너 망극 한 마음 을 놀래다 이다 하나 이다 n 하 거늘 괴히 여기다 그 연고 를 물다 대다 함구 하고 말 이 있다 길이 수천 리라 그것 을 염려 하나 이다 .
s
-----
원통 하다 하다 갈다 아니다 백성 의 의복 을 속이다 재물 을 수탐 하다 가지 고 나 의 의 모라 의논 치 아니다 십일 내 로 길동 을 찾다 종시 거처 를 알 아 지시 하다 인하다 별세 하다 부인 을 대하 여 왈 n 부인 이 옷 을 떨치다 밖 으로 나가시 니 승상 이 연장 구십 에 졸연 득병 하다 추구 월 망일 더욱 중하 여 부인과 장자 길현 을 보다 길동 을 쏘다 하 되다 힘 이 적지 아니다 .
s
-----


In [21]:
import os
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
import urllib.request
import os, re, json, random

In [44]:
# 마르코프 체인 딕셔너리 만들기 --- (※1)
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3: continue
        if len(tmp) > 3: tmp = tmp[1:]
        set_word3(dic, tmp)
        if word == ".":
            tmp = ["@"]
            continue
    return dic
# 딕셔너리에 데이터 등록하기 --- (※2)
def set_word3(dic, s3):
    w1, w2, w3 = s3
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1
# 문장 만들기 --- (※3)
def make_sentence(dic):
    ret = []
    if not "@" in dic: return "no dic" 
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == ".": break
        w1, w2 = w2, w3
    ret = "".join(ret)
    # 띄어쓰기
    params = urllib.parse.urlencode({
        "_callback": "",
        "q": ret
    })
    # 네이버 맞춤법 검사기를 사용합니다.
    # data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy?" + params)
    data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn?" + params)
    data = data.read().decode("utf-8")[1:-2]
    data = json.loads(data)
    data = data["message"]["result"]["html"]
    data = soup = BeautifulSoup(data, "html.parser").getText()
    # 리턴
    return data

def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))
# 문장 읽어 들이기 --- (※4)
toji_file = "data/홍길동전.txt"
dict_file = "markov-toji.json"
if not os.path.exists(dict_file):
    # 텍스트 파일 읽어 들이기
    with open(toji_file, 'r', encoding='utf-8') as f:
        text = str(f.readlines())

    text = text.replace("…", "") # 현재 koNLPy가 …을 구두점으로 잡지 못하는 문제 임시 해결
    text = text.replace("n", "")
    # 형태소 분석
   # twitter = Twitter()
    twitter=Twitter()
    malist = twitter.pos(text, norm=True)
    words = []
    for word in malist:
        # 구두점 등은 대상에서 제외(단 마침표는 포함)
        if not word[1] in ["Punctuation"]:
            words.append(word[0])
        if word[0] == ".":
            words.append(word[0])
    # 딕셔너리 생성
    dic = make_dic(words)
    json.dump(dic, open(dict_file,"w", encoding="utf-8"))
else:
    dic = json.load(open(dict_file,"r"))
# 문장 만들기 --- (※6)
for i in range(3):
    s = make_sentence(dic)
    print(s)
    print("---")

연습을 하고 튀어나올 정도로 거대해졌다.
---
잊지마라유피넬은코스모스Cosmos조화이며헬카네스는모두진지한표정도없었다쾌활하게웃으며말하려하다가다시죽어있는해츨링을가지게되었다.
---
자자 후치와 내 짐만 챙기겠습니다.
---


In [45]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys
with open('data/홍길동전.txt', 'r', encoding='utf-8') as f:
        text = str(f.readlines())
print('코퍼스의 길이: ', len(text))
# 문자를 하나하나 읽어 들이고 ID 붙이기
chars = sorted(list(set(text)))
print('사용되고 있는 문자의 수:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자
# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('학습할 구문의 수:', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
# 모델 구축하기(LSTM)
print('모델을 구축합니다...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('반복 =', iteration)
    model.fit(X, y, batch_size=128) 
    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)
        # 시드를 기반으로 텍스트 자동 생성
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

코퍼스의 길이:  1813923
사용되고 있는 문자의 수: 1644
학습할 구문의 수: 604635
텍스트를 ID 벡터로 변환합니다...


/Users/tj/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/Users/tj/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


모델을 구축합니다...


/Users/tj/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)



--------------------------------------------------
반복 = 1
4724/4724 [==============================] - 678s 143ms/step - loss: 2.5698

--- 다양성 =  0.2
--- 시드 = " 내주었다. 그리고 샌슨의 어머니는 "
 내주었다. 그리고 샌슨의 어머니는 그 말을 내려놓았다. 그 때 그 때 그 때 이루릴이 그 대답을 바라보았다. 그리고 그 때 그 때 그 때 그 때 그 때 이루릴이 그 말을 그것을 가지 않았다. 그 때 그 때 그 때 그 때 그 때 그 때 이루릴이 그 말을 바라보았다. 그리고 그 때 그 때 그 때 그 때 그 때 그 때 그 때 그 때 그 때 그 때 그 때 그 때 이루릴이 그 말을 바라보았다. 그리고 그 때 그 때 그 때 그 때 그 때 이루릴이 그 말을 보\n', '며 말했다.\n', '\n', ' "그렇습니까?"\n', '\n', ' "그렇습니까?"\n', '\n', ' "그렇습니까?"\n', '\n', ' "그렇습니까?"\n', '\n', ' "그렇습니까?"\n', '\n', ' "그렇습니까?"\n', '\n', ' "그렇습니까?"\n', '\n

--- 다양성 =  0.5
--- 시드 = " 내주었다. 그리고 샌슨의 어머니는 "
 내주었다. 그리고 샌슨의 어머니는 그 그 말을 내려놓았다. 그 때 그 때 카알은 말했다.\n', '\n', ' "아, 어떻게 있는 것이라고 하겠습니까?"\n', '\n', ' "그렇습니까?"\n', '\n', ' "그렇다면 네리아가 말이 되고, 그러나 그 말 가지고 있는 것이다. 그 때 그 때 이루릴이 대답했다.\n', '\n', ' "그렇습니까?"\n', '\n', ' "그렇습니까?"\n', '\n', ' "그렇습니다."\n', '\n', ' "그러니까, 우리가 한 거 그렇군요. 저의 불\n', '은 그렇게 그렇게  있는 것은 아니고, 그 \n', '아마 살아들은 거야? 하지만 이 사람들이 사라지지 않겠습니까?"

KeyboardInterrupt: 